# Usando RNA para Regressão com 3 saídas

Uso da Base de dados Video Game Sales with Ratings do Kaggle, para previsão das vendas na America do norte, no japão e na Europa através de uma Rede Neural Artificial.

## Importação das bibliotecas e dos Dados

In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model 

In [2]:
base = pd.read_csv('../datasets/games.csv')

## Pre Processamento de Dados

In [3]:
base = base.drop('Other_Sales', axis=1)
base = base.drop('Global_Sales', axis=1)
base = base.drop('Developer', axis=1)

In [4]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 13 columns):
Name               16717 non-null object
Platform           16719 non-null object
Year_of_Release    16450 non-null float64
Genre              16717 non-null object
Publisher          16665 non-null object
NA_Sales           16719 non-null float64
EU_Sales           16719 non-null float64
JP_Sales           16719 non-null float64
Critic_Score       8137 non-null float64
Critic_Count       8137 non-null float64
User_Score         10015 non-null object
User_Count         7590 non-null float64
Rating             9950 non-null object
dtypes: float64(7), object(6)
memory usage: 1.7+ MB


In [5]:
pd.DataFrame({'nomes': base.columns, 'tipos': base.dtypes, 'NA #': base.isna().sum(),
                                   'NA %': (base.isna().sum() / base.shape[0]) * 100})

,nomes,tipos,NA #,NA %
Name,Name,object,2,0.011962
Platform,Platform,object,0,0.000000
Year_of_Release,Year_of_Release,float64,269,1.608948
Genre,Genre,object,2,0.011962
Publisher,Publisher,object,54,0.322986
NA_Sales,NA_Sales,float64,0,0.000000
EU_Sales,EU_Sales,float64,0,0.000000
JP_Sales,JP_Sales,float64,0,0.000000
Critic_Score,Critic_Score,float64,8582,51.330821
Critic_Count,Critic_Count,float64,8582,51.330821


In [6]:
base['Critic_Score'] = base['Critic_Score'].fillna(base['Critic_Score'].mean())
base['Critic_Count'] = base['Critic_Count'].fillna(base['Critic_Count'].mean())
base['User_Count'] = base['User_Count'].fillna(base['User_Count'].mean())

In [7]:
base = base.dropna(axis=0)

In [8]:
base.shape

(9705, 13)

In [9]:
base = base.loc[base['NA_Sales'] > 1]
base = base.loc[base['EU_Sales'] > 1]

In [10]:
base['Name'].value_counts()

Call of Duty: Advanced Warfare     4
Grand Theft Auto V                 4
The Elder Scrolls V: Skyrim        3
Call of Duty: Ghosts               3
Assassin's Creed IV: Black Flag    3
                                  ..
Skylanders: Spyro's Adventure      1
Need for Speed: Most Wanted        1
Kinect Sports                      1
Wii Sports                         1
Tomb Raider                        1
Name: Name, Length: 230, dtype: int64

In [11]:
base = base[base['User_Score'] != 'tbd']

In [12]:
nome_jogos = base.Name
base = base.drop('Name', axis=1)

In [82]:
previsores = base.iloc[:, [0,1,2,3,7,8,9,10,11]]

In [106]:
venda_na = base.iloc[:, 4]
venda_eu = base.iloc[:, 5]
venda_jp = base.iloc[:, 6]

In [95]:
previsores.dtypes

Platform            object
Year_of_Release    float64
Genre               object
Publisher           object
Critic_Score       float64
Critic_Count       float64
User_Score         float64
User_Count         float64
Rating              object
dtype: object

In [90]:
previsores.iloc[:,6] = previsores.iloc[:,6].astype('float64')

/home/capizani/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [94]:
type(previsores.iloc[:,1][0])

numpy.float64

In [105]:
X = pd.concat([previsores, pd.get_dummies(previsores.iloc[:,[0,2,3,8]])], axis=1)\
        .drop(['Platform','Genre','Publisher','Rating'], axis=1)

In [76]:
previsores = np.concatenate((previsores2,np.asarray(previsores[:,[1,4,5,6,7]])),axis=1)

In [77]:
print(type(previsores))
print(type(venda_eu))
print(type(venda_jp))
print(type(venda_na))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [78]:
previsores.shape

(264, 61)

## Construção da RNA

In [110]:
camada_entrada = Input(shape=(61,))
camada_oculta1 = Dense(units= 32, activation ='sigmoid')(camada_entrada)
camada_oculta2 = Dense(units= 32, activation ='sigmoid')(camada_oculta1)
camada_saida1 = Dense(units= 1, activation ='linear')(camada_oculta2)
camada_saida2 = Dense(units= 1, activation ='linear')(camada_oculta2)
camada_saida3 = Dense(units= 1, activation ='linear')(camada_oculta2)

In [111]:
regressor = Model(inputs = camada_entrada,
                 outputs = [camada_saida1, camada_saida2, camada_saida3])
regressor.compile(optimizer='adam', loss='mse')

In [114]:
regressor.fit(X, [venda_na, venda_eu, venda_jp], epochs=5000, batch_size=64)

Epoch 1/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3381 - dense_22_loss: 11.5900 - dense_23_loss: 5.6882 - dense_24_loss: 1.0599
Epoch 2/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.3482 - dense_22_loss: 11.6050 - dense_23_loss: 5.6846 - dense_24_loss: 1.0587
Epoch 3/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4444 - dense_22_loss: 11.6612 - dense_23_loss: 5.7220 - dense_24_loss: 1.0612
Epoch 4/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4319 - dense_22_loss: 11.6504 - dense_23_loss: 5.7180 - dense_24_loss: 1.0635
Epoch 5/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4069 - dense_22_loss: 11.6345 - dense_23_loss: 5.7093 - dense_24_loss: 1.0631
Epoch 6/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3778 - dense_22_loss: 11.6137 - dense_23_loss: 5.7010 - dense_24_loss: 1.0631
Epoch 7/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3515 - 

5/5 [==============================] - 0s 1ms/step - loss: 18.3228 - dense_22_loss: 11.5785 - dense_23_loss: 5.6855 - dense_24_loss: 1.0587
Epoch 55/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3309 - dense_22_loss: 11.5861 - dense_23_loss: 5.6886 - dense_24_loss: 1.0561
Epoch 56/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.3814 - dense_22_loss: 11.6095 - dense_23_loss: 5.7040 - dense_24_loss: 1.0678
Epoch 57/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3340 - dense_22_loss: 11.5857 - dense_23_loss: 5.6863 - dense_24_loss: 1.0620
Epoch 58/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3476 - dense_22_loss: 11.5912 - dense_23_loss: 5.6898 - dense_24_loss: 1.0666
Epoch 59/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3498 - dense_22_loss: 11.5919 - dense_23_loss: 5.6916 - dense_24_loss: 1.0663
Epoch 60/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3466 - dense_2

5/5 [==============================] - 0s 2ms/step - loss: 18.3066 - dense_22_loss: 11.5683 - dense_23_loss: 5.6779 - dense_24_loss: 1.0604
Epoch 107/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3127 - dense_22_loss: 11.5726 - dense_23_loss: 5.6797 - dense_24_loss: 1.0604
Epoch 108/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.3060 - dense_22_loss: 11.5680 - dense_23_loss: 5.6788 - dense_24_loss: 1.0592
Epoch 109/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3428 - dense_22_loss: 11.5953 - dense_23_loss: 5.6894 - dense_24_loss: 1.0581
Epoch 110/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3378 - dense_22_loss: 11.5918 - dense_23_loss: 5.6895 - dense_24_loss: 1.0565
Epoch 111/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3231 - dense_22_loss: 11.5814 - dense_23_loss: 5.6834 - dense_24_loss: 1.0583
Epoch 112/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.3138 - d

5/5 [==============================] - 0s 1ms/step - loss: 18.4262 - dense_22_loss: 11.6278 - dense_23_loss: 5.7198 - dense_24_loss: 1.0786
Epoch 159/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3279 - dense_22_loss: 11.5519 - dense_23_loss: 5.6975 - dense_24_loss: 1.0785
Epoch 160/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3369 - dense_22_loss: 11.5603 - dense_23_loss: 5.6993 - dense_24_loss: 1.0774
Epoch 161/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3819 - dense_22_loss: 11.5945 - dense_23_loss: 5.7109 - dense_24_loss: 1.0765
Epoch 162/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3823 - dense_22_loss: 11.5953 - dense_23_loss: 5.7101 - dense_24_loss: 1.0769
Epoch 163/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3688 - dense_22_loss: 11.5846 - dense_23_loss: 5.7068 - dense_24_loss: 1.0774
Epoch 164/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.3378 - d

5/5 [==============================] - 0s 1ms/step - loss: 18.2967 - dense_22_loss: 11.5330 - dense_23_loss: 5.6879 - dense_24_loss: 1.0758
Epoch 211/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3093 - dense_22_loss: 11.5421 - dense_23_loss: 5.6917 - dense_24_loss: 1.0755
Epoch 212/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3500 - dense_22_loss: 11.5710 - dense_23_loss: 5.7021 - dense_24_loss: 1.0769
Epoch 213/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3427 - dense_22_loss: 11.5669 - dense_23_loss: 5.6993 - dense_24_loss: 1.0765
Epoch 214/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3461 - dense_22_loss: 11.5712 - dense_23_loss: 5.6987 - dense_24_loss: 1.0762
Epoch 215/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3064 - dense_22_loss: 11.5389 - dense_23_loss: 5.6915 - dense_24_loss: 1.0761
Epoch 216/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.3114 - d

...

5/5 [==============================] - 0s 1ms/step - loss: 18.4259 - dense_22_loss: 11.6253 - dense_23_loss: 5.6804 - dense_24_loss: 1.1202
Epoch 4787/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4201 - dense_22_loss: 11.6232 - dense_23_loss: 5.6776 - dense_24_loss: 1.1192
Epoch 4788/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4193 - dense_22_loss: 11.6219 - dense_23_loss: 5.6782 - dense_24_loss: 1.1192
Epoch 4789/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4318 - dense_22_loss: 11.6298 - dense_23_loss: 5.6814 - dense_24_loss: 1.1206
Epoch 4790/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4319 - dense_22_loss: 11.6287 - dense_23_loss: 5.6824 - dense_24_loss: 1.1208
Epoch 4791/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4392 - dense_22_loss: 11.6352 - dense_23_loss: 5.6849 - dense_24_loss: 1.1191
Epoch 4792/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.43

Epoch 4838/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4203 - dense_22_loss: 11.6191 - dense_23_loss: 5.6820 - dense_24_loss: 1.1192
Epoch 4839/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4216 - dense_22_loss: 11.6195 - dense_23_loss: 5.6826 - dense_24_loss: 1.1195
Epoch 4840/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4270 - dense_22_loss: 11.6243 - dense_23_loss: 5.6835 - dense_24_loss: 1.1192
Epoch 4841/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4274 - dense_22_loss: 11.6251 - dense_23_loss: 5.6834 - dense_24_loss: 1.1189
Epoch 4842/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4147 - dense_22_loss: 11.6149 - dense_23_loss: 5.6808 - dense_24_loss: 1.1190
Epoch 4843/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4176 - dense_22_loss: 11.6159 - dense_23_loss: 5.6825 - dense_24_loss: 1.1192
Epoch 4844/5000
5/5 [==============================] - 0s 2ms/st

5/5 [==============================] - 0s 2ms/step - loss: 18.4149 - dense_22_loss: 11.6110 - dense_23_loss: 5.6801 - dense_24_loss: 1.1238
Epoch 4891/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4262 - dense_22_loss: 11.6158 - dense_23_loss: 5.6866 - dense_24_loss: 1.1237
Epoch 4892/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4209 - dense_22_loss: 11.6128 - dense_23_loss: 5.6842 - dense_24_loss: 1.1239
Epoch 4893/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4236 - dense_22_loss: 11.6145 - dense_23_loss: 5.6849 - dense_24_loss: 1.1241
Epoch 4894/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4131 - dense_22_loss: 11.6081 - dense_23_loss: 5.6808 - dense_24_loss: 1.1242
Epoch 4895/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4177 - dense_22_loss: 11.6107 - dense_23_loss: 5.6828 - dense_24_loss: 1.1242
Epoch 4896/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.42

5/5 [==============================] - 0s 2ms/step - loss: 18.4252 - dense_22_loss: 11.6112 - dense_23_loss: 5.6928 - dense_24_loss: 1.1211
Epoch 4943/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4269 - dense_22_loss: 11.6120 - dense_23_loss: 5.6939 - dense_24_loss: 1.1210
Epoch 4944/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4226 - dense_22_loss: 11.6096 - dense_23_loss: 5.6919 - dense_24_loss: 1.1212
Epoch 4945/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4231 - dense_22_loss: 11.6105 - dense_23_loss: 5.6917 - dense_24_loss: 1.1209
Epoch 4946/5000
5/5 [==============================] - 0s 3ms/step - loss: 18.4246 - dense_22_loss: 11.6129 - dense_23_loss: 5.6915 - dense_24_loss: 1.1202
Epoch 4947/5000
5/5 [==============================] - 0s 3ms/step - loss: 18.4347 - dense_22_loss: 11.6193 - dense_23_loss: 5.6943 - dense_24_loss: 1.1212
Epoch 4948/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.42

5/5 [==============================] - 0s 2ms/step - loss: 18.4361 - dense_22_loss: 11.6210 - dense_23_loss: 5.6941 - dense_24_loss: 1.1211
Epoch 4995/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.4517 - dense_22_loss: 11.6301 - dense_23_loss: 5.7002 - dense_24_loss: 1.1214
Epoch 4996/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4562 - dense_22_loss: 11.6335 - dense_23_loss: 5.7009 - dense_24_loss: 1.1218
Epoch 4997/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.4884 - dense_22_loss: 11.6541 - dense_23_loss: 5.7102 - dense_24_loss: 1.1241
Epoch 4998/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.6143 - dense_22_loss: 11.7284 - dense_23_loss: 5.7602 - dense_24_loss: 1.1256
Epoch 4999/5000
5/5 [==============================] - 0s 1ms/step - loss: 18.6284 - dense_22_loss: 11.7376 - dense_23_loss: 5.7626 - dense_24_loss: 1.1283
Epoch 5000/5000
5/5 [==============================] - 0s 2ms/step - loss: 18.57